In [1]:
import cv2 as cv
import numpy as np
import epfl_data2 as epfl_data
import matplotlib.pyplot as plt
from skimage.feature import hog as HoG
from sklearn.preprocessing import normalize
from sklearn.utils import shuffle

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [2]:
def flip_angle(angle):
    if angle == 180:
        return 0
    elif angle == 0:
        return 180
    else:
        return 360-angle


data = epfl_data.Data(1,6)

training_images = np.zeros((1327,11016))
training_images_f = np.zeros((1327,11016))
test_images = np.zeros((972,11016))
training_tgts = []
training_tgts_f = []
test_tgts = []

height = 250
width = 376

for d in range(len(data.samples[0])*2):
    image = cv.imread(data.samples[0][d])
    image = image[28:212, 30:348]
    image = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    f_image = np.fliplr(image)
    if d <= 1326:
        image = HoG(image, orientations=8, pixels_per_cell=(16,16))
        image = normalize(image)
        f_image = HoG(f_image, orientations=8, pixels_per_cell=(16,16))
        f_image = normalize(image)
        training_images[d] = image
        training_tgts.append(np.deg2rad(data.samples[1][d]))
        training_images_f[d] = f_image
        training_tgts_f.append(np.deg2rad(flip_angle(data.samples[1][d])))
    else:
        if d-1326 == 972:
            break
        else:
            image = HoG(image, orientations=8, pixels_per_cell=(16, 16))
            image = normalize(image)
            test_images[d-1326] = image
            test_tgts.append(np.deg2rad(data.samples[1][d]))


train = np.append(training_images, training_images_f,axis=0)
for i in range(len(training_tgts)):
    training_tgts.append(training_tgts_f[i])

(1327, 11016)
(2654, 11016)


In [3]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers.convolutional import Conv1D

Using TensorFlow backend.


In [7]:
model = Sequential()
model.add(Dense(1000, input_dim=11016, activation='relu'))
# model.add(Dropout(.5))
model.add(Dense(100, activation='relu'))
# model.add(Dropout(.5))
model.add(Dense(10, activation='relu'))
# model.add(Dropout(.5))
model.add(Dense(10, activation='relu'))
# model.add(Dropout(.5))
model.add(Dense(1))
# model.add(Dropout(.5))

model.compile(optimizer="adam", loss='mean_squared_error', metrics=['accuracy'])

model.fit(train, np.asarray(training_tgts), batch_size=32, epochs=20, verbose=1)

score = model.evaluate(test_images[:-1], np.asarray(test_tgts), verbose=1)
print("\nSCORE:", score)

model.summary()

predictions = model.predict(test_images[:-1])

print("PREDICTIONS:", predictions.T)
print("ACTUAL:", test_tgts)

Epoch 1/20
2654/2654 [==============================] - 38s - loss: 3.8377 - acc: 0.0000e+00    
Epoch 2/20
2654/2654 [==============================] - 37s - loss: 3.4443 - acc: 0.0000e+00    
Epoch 3/20
2654/2654 [==============================] - 36s - loss: 3.4324 - acc: 0.0000e+00    
Epoch 4/20
2654/2654 [==============================] - 37s - loss: 3.4641 - acc: 0.0000e+00    
Epoch 5/20
2654/2654 [==============================] - 36s - loss: 3.4087 - acc: 0.0000e+00    
Epoch 6/20
2654/2654 [==============================] - 36s - loss: 3.4376 - acc: 0.0000e+00    
Epoch 7/20
2654/2654 [==============================] - 36s - loss: 3.4212 - acc: 0.0000e+00    
Epoch 8/20
2654/2654 [==============================] - 36s - loss: 3.3881 - acc: 0.0000e+00    
Epoch 9/20
2654/2654 [==============================] - 41s - loss: 3.4187 - acc: 0.0000e+00    
Epoch 10/20
2654/2654 [==============================] - 36s - loss: 3.3989 - acc: 0.0000e+00    
Epoch 11/20
2654/2654 [======

In [9]:
print(predictions[10:20].T)
print(test_tgts[10:20])

[[ 2.27812481  2.20622826  2.09912515  2.10791016  2.18459177  2.27436161
   2.38084888  2.51445436  2.50551176  2.51277924]]
[0.089759790102565448, 0.029919930034188649, 6.2532653771453983, 6.1934255170770216, 6.1934255170770216, 6.133585657008644, 6.0139059368718897, 5.8343863566667595, 5.7745464965983819, 5.7147066365300052]
